In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-207427
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-207427


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "rtw-compute-sdk"

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

In [23]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.01, 0.1, 1),
        '--max_iter': choice(10, 20, 50, 100, 250)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', 
                      command=['python', 'train.py'],
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=4,
                                     max_concurrent_runs=4)

In [24]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_14037c89-03be-46a0-b623-172677aa9ec8
Web View: https://ml.azure.com/runs/HD_14037c89-03be-46a0-b623-172677aa9ec8?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-207427/workspaces/quick-starts-ws-207427&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-09-15T01:01:50.866063][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-09-15T01:01:51.6317120Z][SCHEDULER][INFO]Scheduling job, id='HD_14037c89-03be-46a0-b623-172677aa9ec8_0' 
[2022-09-15T01:01:51.7697418Z][SCHEDULER][INFO]Scheduling job, id='HD_14037c89-03be-46a0-b623-172677aa9ec8_1' 
[2022-09-15T01:01:51.8502453Z][SCHEDULER][INFO]Scheduling job, id='HD_14037c89-03be-46a0-b623-172677aa9ec8_2' 
[2022-09-15T01:01:51.919813][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-09-15T01:01:51.9678001Z][SCHEDULER][INFO]Scheduling job, id='HD_14037c89-03be-46a0-b623-172677aa9e

{'runId': 'HD_14037c89-03be-46a0-b623-172677aa9ec8',
 'target': 'rtw-compute-sdk',
 'status': 'Completed',
 'startTimeUtc': '2022-09-15T01:01:50.242095Z',
 'endTimeUtc': '2022-09-15T01:07:24.183523Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'cc6d711c-833f-4ff4-8d1e-88121a8dc98e',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': '15',
  'score': '0.9103186646433991',
  'best_child_run_id': 'HD_14037c89-03be-46a0-b623-172677aa9ec8_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_14037c89-03be-46a0-b623-172677aa9ec8_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCli

In [32]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

In [30]:
best_run_metrics

{'Regularization Strength:': 1.0,
 'Max iterations:': 100,
 'Accuracy': 0.9103186646433991}

In [31]:
type(best_run)

azureml.core.run.Run

### AutoML

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [14]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [18]:
from azureml.train.automl import AutoMLConfig
import pandas as pd

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=pd.concat([x_train, y_train], axis=1),
    label_column_name='y',
    n_cross_validations=4)

In [19]:
# Submit your automl run
auto_run = exp.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_fa100e1d-2651-4dd3-8b08-45cca4dedf67,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


In [ ]:
# Retrieve and save your best automl model.
auto_run.get_metrics()

{'average_precision_score_macro': 0.8266277197670922,
 'precision_score_macro': 0.7980517230982558,
 'recall_score_micro': 0.9174127465857359,
 'recall_score_macro': 0.7696880817608348,
 'precision_score_weighted': 0.9138063906218572,
 'recall_score_weighted': 0.9174127465857359,
 'average_precision_score_micro': 0.9813164241485699,
 'log_loss': 0.1917008480029479,
 'weighted_accuracy': 0.9542897015571483,
 'norm_macro_recall': 0.5393761635216696,
 'f1_score_weighted': 0.9153149615367752,
 'accuracy': 0.9174127465857359,
 'f1_score_micro': 0.9174127465857359,
 'AUC_weighted': 0.9472071754952476,
 'balanced_accuracy': 0.7696880817608348,
 'precision_score_micro': 0.9174127465857359,
 'f1_score_macro': 0.782749226490939,
 'AUC_micro': 0.980533680957721,
 'average_precision_score_weighted': 0.9556766673767547,
 'AUC_macro': 0.9472071754952476,
 'matthews_correlation': 0.5669552527152837}

In [33]:
best_auto_run, fitted_auto_model = auto_run.get_output()

In [34]:
best_auto_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_fa100e1d-2651-4dd3-8b08-45cca4dedf67_25,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [35]:
best_auto_run.get_metrics()

{'matthews_correlation': 0.5669552527152837,
 'AUC_macro': 0.9472071754952476,
 'average_precision_score_weighted': 0.9556766673767547,
 'weighted_accuracy': 0.9542897015571483,
 'AUC_weighted': 0.9472071754952476,
 'recall_score_macro': 0.7696880817608348,
 'precision_score_weighted': 0.9138063906218572,
 'norm_macro_recall': 0.5393761635216696,
 'recall_score_weighted': 0.9174127465857359,
 'recall_score_micro': 0.9174127465857359,
 'balanced_accuracy': 0.7696880817608348,
 'average_precision_score_macro': 0.8266277197670922,
 'AUC_micro': 0.980533680957721,
 'precision_score_micro': 0.9174127465857359,
 'log_loss': 0.1917008480029479,
 'f1_score_micro': 0.9174127465857359,
 'f1_score_macro': 0.782749226490939,
 'precision_score_macro': 0.7980517230982558,
 'accuracy': 0.9174127465857359,
 'f1_score_weighted': 0.9153149615367752,
 'average_precision_score_micro': 0.9813164241485699,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_fa100e1d-2651-4dd3-8b08-45cca4dedf67_25

In [37]:
best_auto_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'local\',\'subscription_id\':\'3d1a56d2-7c81-4118-9790-f85d1acf0c77\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_fa100e1d-2651-4dd3-8b08-45cca4dedf67_25","experiment_name":null,"workspace_name":"quick-starts-ws-207427","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group_name":"aml-quickstarts-207427"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '25',
 '_azureml.ComputeTargetType': 'local',
 '_aml_system_scenario_identification': 'Local.Child',
 'ru

In [39]:
# Clean up compute cluster
cpu_cluster.delete()